## Back Test Data Generation

In [1]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
pd.get_option("display.max_columns", None)

20

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_k_similar_candles(candle_id, dataset, k=4):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'auto') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
#                            dataset.iloc[indices.flatten()[i]]['O-H'],dataset.iloc[indices.flatten()[i]]['O-L'],dataset.iloc[indices.flatten()[i]]['O-C'],dataset.iloc[indices.flatten()[i]]['H-L'],dataset.iloc[indices.flatten()[i]]['H-C'],dataset.iloc[indices.flatten()[i]]['L-C'],
                            dataset.iloc[indices.flatten()[i]]['col_1'],
                            dataset.iloc[indices.flatten()[i]]['col_2'],
                            dataset.iloc[indices.flatten()[i]]['col_3'],
#                            dataset.iloc[indices.flatten()[i]]['col_4'],
#                            dataset.iloc[indices.flatten()[i]]['col_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_5'],
                            dataset.iloc[indices.flatten()[i]]['F_SMA_10'],
#                            dataset.iloc[indices.flatten()[i]]['F_SMA_20']
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
#                      'O-H','O-L','O-C','H-L','H-C','L-C',
                      'col_1',
                      'col_2',
                      'col_3',
#                      'col_4',
#                      'col_5',
                      'F_SMA_5',
                      'F_SMA_10',
#                      'F_SMA_20',
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [5]:
filename = 'USD_CAD_H4.csv'
data = pd.read_csv(filename)

In [6]:
data.columns

Index(['Date', 'Time', 'f_time', 'julian_date', 'Weekday', 'Weekday_Name',
       'UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close', 'SMA_5', 'SMA_10',
       'SMA_20', 'F_SMA_5', 'F_SMA_10', 'F_SMA_20', 'O-H', 'O-L', 'O-C', 'H-L',
       'H-C', 'L-C', 'Direction', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5'],
      dtype='object')

In [7]:
data.head()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-09-15,01:00:00,1,258,1,Tuesday,2015-09-15T01:00:00.000000000Z,2597,1.32521,1.32638,...,-0.00079,0.00136,0.00038,-0.00098,1,-0.00079,0.00038,0.00019,0.00111,0.00088
1,2015-09-15,05:00:00,5,258,1,Tuesday,2015-09-15T05:00:00.000000000Z,5316,1.32600,1.32664,...,0.00168,0.00254,0.00232,-0.00022,0,0.00168,0.00232,0.00190,-0.00079,0.00111
2,2015-09-15,09:00:00,9,258,1,Tuesday,2015-09-15T09:00:00.000000000Z,6747,1.32432,1.32605,...,-0.00098,0.00336,0.00075,-0.00261,1,-0.00098,0.00075,0.00163,0.00168,-0.00079
3,2015-09-15,13:00:00,13,258,1,Tuesday,2015-09-15T13:00:00.000000000Z,8374,1.32530,1.32728,...,-0.00067,0.00375,0.00131,-0.00244,1,-0.00067,0.00131,0.00177,-0.00098,0.00168
4,2015-09-15,17:00:00,17,258,1,Tuesday,2015-09-15T17:00:00.000000000Z,5375,1.32601,1.32698,...,0.00080,0.00348,0.00177,-0.00171,0,0.00080,0.00177,0.00251,-0.00067,-0.00098


## Selecting 1000 Random Candles

In [9]:
Test_Candle = [6588, 4779, 6286, 1205, 2823, 5756, 8321, 1810,  802, 1479, 5824,
               7199, 5209, 2539, 5687, 4733, 5952, 4035, 8125, 2683, 8425, 5455,
               4584, 3795, 8412, 8765, 6830, 3859,  852, 2728, 8516, 2794, 6351,
               6565, 2120, 8666, 3258,  526, 1423, 3286, 9089, 2199, 3652, 9160,
               7704, 2174, 5186, 8200, 2948, 2663, 9170, 8718, 2666, 1407, 3606,
               2790, 6293, 2480, 2245,  650, 7355, 8157, 5573, 1075, 8973, 1160,
               3202, 7821, 2524,  782, 4487, 3208, 3685, 8147, 2479, 8747,  616,
               9438, 1056, 6287, 8969, 7735, 7520, 9166, 5563, 5038, 8299, 8429,
               8654, 1944, 5243, 9364, 1327, 2457, 2657,  697, 7846, 1045, 5062,
               6524,  312, 8947, 2111, 2707, 1723, 2840, 1742, 4237, 5069, 7511,
               1027, 4917, 4120, 7105, 5018, 8440,  504, 8038, 5463, 3659, 5980,
               3089, 7401,  820, 7790, 6710, 3074, 6408, 4575, 7238, 1935, 5109,
               8531,  595, 4274, 2204, 8925, 8321,  554, 8721, 6160, 6953, 6737,
               3271, 2455, 9299, 3816, 1177, 8808, 2652, 2296, 9173, 1865, 6868,
               2777, 1716, 5153, 7363, 1452, 2136, 2197, 8505, 1683, 8026, 5698,
               7672, 6586, 6975, 7280, 9355, 1604, 8530, 3165, 1985, 1492, 7075,
               4014, 5584, 8898, 8141, 9218, 1565, 3323, 8071, 3663,  929, 1007,
                139, 8897, 4972, 6209, 7346, 4628,  208, 2553, 1904, 9156, 8473,
               9291, 5084, 3496, 6304, 5382, 8450, 2770, 8408, 8295,  173, 1563,
                636, 1560,  466, 1870, 8617, 7220, 5927, 8322, 5874, 6502, 9498,
               3639, 4599, 3606, 7152, 5568, 6427, 6582, 5497, 8057, 1750, 6868,
               5616, 6998, 6386, 5693, 2438, 2134, 8656, 9256,  862, 1591, 2008,
               8297, 8066, 5299, 9092, 8415, 6620, 7634, 7082, 4327, 7346,  325,
               6075,  970, 8230, 2655, 1483,  840, 1654, 5974, 5558, 3756,  493,
               1739, 3432, 3633, 4197, 1755, 8569, 6272, 3878, 4100, 3972, 3142,
               8660, 5057, 5735, 7944, 7905, 6023,  455, 7719, 8353, 2939,  923,
               4587, 6084, 9357, 3343, 1069,  383, 3530, 1505, 7174, 5753,    9,
               8175,  662, 5012, 5723, 4752, 2472,  436, 9012, 1670, 2069, 3799,
                571, 8490, 7781,  798, 3396, 1412, 7718,  614, 6495, 6313, 6216,
               6527, 5231, 7510, 1180, 3616, 7445, 1524, 7885, 8610, 8781, 6415,
               8494, 4071, 4795, 5759, 7954, 9398,  953, 6147, 3280, 7355, 7663,
               6467, 5617, 7214, 5567, 5156, 5845, 6975, 6199, 7890, 9131, 8905,
               2654, 6363, 6678, 2494, 8150, 7168, 5170, 2548, 5290,  211, 3108,
               7926, 5795, 3969, 4432, 4911, 7899, 4211, 1357, 9164, 9413,  794,
               4498,  450, 6563, 6672, 4729, 5965, 1368, 2240, 4219, 3691,  656,
               1380, 2763, 1351, 7541,  500, 2679, 6820,  521, 5355, 3318, 6926,
               7452, 6420, 2094, 2153, 7141, 6416, 5799, 5034,  712, 3202, 8174,
               3045,  674, 6736, 9110, 4396, 3037, 1059, 6022,  343, 5443, 3364,
               4590, 7762, 5594, 7097, 5159, 1488, 6712,  648, 8597, 1334, 1159,
               8128, 5495, 3229, 2520, 6534, 4185, 5651, 2775, 1388, 3291,  374,
               8570,  683, 3767, 8497, 1841, 4749, 1354, 5635, 4736, 2885,  100,
               3491, 5911, 1808, 4319, 2203, 9179, 6263,  464, 4712, 7970, 1783,
               8240, 7711,  600, 8680, 6457, 6267, 8387, 7115, 5776, 4132, 5149,
               7709, 5947, 2840, 1271, 5191,  141,  266, 7438, 6669, 2507, 5539,
               8859, 1180, 6043, 4339, 7716, 8195, 2570, 4656,  566, 7054, 3135,
               5596, 1004, 1548, 5158,  747, 1031, 3494, 6625, 3878, 9344, 4505,
               5508, 7830, 8880,  689, 2041, 3992, 5690, 2486, 1536, 1039, 4032,
               8266, 7915, 8218, 4928, 6039, 4480, 9361, 3150, 8234, 1993, 4821,
               5546, 1962, 2326, 9151, 1341, 7596, 2964, 5437, 5780,  747, 8895,
               7869, 2837, 5069, 5509, 6374,  647, 3930, 7898, 5889, 7835, 8565,
               4692, 3751, 5040, 4502, 8148, 4032,  828, 8933, 4761, 9233, 3005,
               8575, 6978, 4164, 4291, 8357, 3756, 4036, 3435, 1426, 5740, 5684,
               5667, 3205, 5102, 4740, 7981, 3759, 7485, 8003, 4702,  191, 7747,
               9476, 5334, 3773, 3642, 3859, 1032, 9395,  516, 7099,  686,   66,
                177, 8936, 2169,  849, 3206, 9066, 1906, 5850, 6382, 5423, 8535,
               8046, 7168, 6573, 3736, 1804, 1139, 8209,  893, 5667, 2597, 8130,
               3307, 5089, 1136, 3053, 4432, 9007, 9196, 4998,   82,   29, 2761,
               8144, 4857, 9116, 9000, 7162, 9105, 8622, 4362, 6816, 4850, 9030,
               2820, 9438, 8681, 7891, 1183, 5130, 8844, 7548, 8353, 5489, 2972,
               3550, 6857, 4453, 1964, 4043, 8372, 7243, 5721, 3388, 4258, 4691,
               1336, 6358, 3488, 6070, 2156, 1121, 2105, 5710, 2304, 4780, 7508,
                879, 7800, 9043, 6477, 8917, 1892,  243, 5787,  943, 6276, 9144,
               4817, 6901, 2602, 6401, 4078, 8975,  744, 9129,  438,  155, 2027,
               9146, 4231, 5665, 8362, 6191, 2600, 7863, 2426, 8760, 2653, 4254,
               4649, 9394, 3656, 7578, 4697, 4561, 9394, 8188, 5236,  832, 4406,
               6299, 2940, 7756, 9437, 8730, 7296, 2460, 4960, 8294, 8906, 5916,
               6810, 2395, 3275,  105, 8352, 1483, 3257, 3839, 6011, 3441, 8081,
                962, 4622, 7938, 6946, 6404, 5987, 1608, 4870, 5452,  146, 1852,
               6319,  621,  592, 3865, 6948,  570, 3238, 3881, 6974, 6581, 2982,
               8095, 7619, 7223, 5109, 5478, 7054, 8254, 5156, 4440, 5402, 9435,
               2204, 4374, 2870, 4111, 8129, 1873, 5819, 1108, 6716, 6886, 4775,
                183, 2318, 7881, 3357, 7800,  594,  500, 5622, 3911, 6245, 7481,
                 51, 3529, 5591,  588, 2503, 8433,   69,  534, 5463, 6578,  366,
               4723,  733,  876, 9193, 5247,  446, 2801, 2095, 6108, 4402,  556,
               3693, 4753, 2954, 1152, 4537, 4766, 1599, 5452, 6564, 3707, 6533,
               8821, 3621, 6977, 7721,  193, 8846, 9542, 8263, 3124, 2774, 5465,
               3533, 3152, 7890,  657, 9225, 7603,  397,  864, 9254, 3212, 6095,
               1835, 9354,  920,  927, 8064, 7996, 1254, 5487, 6160, 1372,  467,
               6393, 1814, 4001, 8012, 4869, 9216, 9249,  146,  357, 3564,  909,
               8222, 5781, 7550, 4647, 2668, 4445, 3065, 5829, 1455, 4627, 2573,
               2344, 5724,  707, 5555, 4731, 3680, 5109, 4972, 6434, 2244, 6942,
               3495, 3137, 7563, 8969, 2271, 9186, 5801, 4682, 1411, 5603, 1281,
               8043, 1068, 3593, 8823, 3928, 7832, 4783,  212, 3930, 7108, 6030,
               6482, 1916, 6315, 3893, 7104, 6103, 4403, 6571, 9373, 5435, 5075,
               9543, 7330, 4919, 8791, 6982, 4255,  953, 2822, 5494, 3719, 5180,
               4368, 1683, 8983, 3997, 8289, 7996, 4064, 9200, 2214, 4550, 1331,
               1741, 5893, 2798, 8255, 6362, 8069, 5009, 8059, 7443, 3458, 7501,
                 95, 5108, 6204, 1410, 1031, 6383, 7052, 8959, 5921, 1892, 5672,
               1101, 5891,  992, 5362, 2626, 6403, 3684, 6776, 1116, 1190, 3618,
                336, 8157, 7411, 7500, 5932, 4250, 3448, 3937, 5170,  274, 4345,
               5648, 2698, 3396,  462, 1274, 8119, 9441, 1832, 8420, 9092
              ]

In [10]:
data.shape

(9639, 30)

In [11]:
data.head(2)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,...,O-C,H-L,H-C,L-C,Direction,col_1,col_2,col_3,col_4,col_5
0,2015-09-15,01:00:00,1,258,1,Tuesday,2015-09-15T01:00:00.000000000Z,2597,1.32521,1.32638,...,-0.00079,0.00136,0.00038,-0.00098,1,-0.00079,0.00038,0.00019,0.00111,0.00088
1,2015-09-15,05:00:00,5,258,1,Tuesday,2015-09-15T05:00:00.000000000Z,5316,1.32600,1.32664,...,0.00168,0.00254,0.00232,-0.00022,0,0.00168,0.00232,0.00190,-0.00079,0.00111


# <font color='red'>CANDLE LOOP</font>

In [12]:
%%time

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              'Current_Market_Stoploss':[],
                              'Rec1':[],
                              'Rec1_P':[],
                              'Rec2':[],
                              'Rec2_P':[],
                              'Rec3':[],
                              'Rec3_P':[]
                             })

for candle_no in Test_Candle:

    #print('candle number is = ',candle_no)
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+3]
    #print('data is = ', data)
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred).round(2)*100
    #print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market='Bullish'
        Current_Market_Stoploss= 0
    else:
        Current_Market = 'Bearish'
        Current_Market_Stoploss= 0 
    
    data = pd.read_csv(filename)
    data = data[['col_1','col_2','col_3','F_SMA_5','F_SMA_10']]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    predicted_output = []
    for indice in indices[1:4]:
             
        Predicted_Market_Fit =0
        Predicted_Trade=''
    
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+3]

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        Y = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y)
        y_pred = linear_regressor.predict(X)

        Predicted_Market_Fit= r2_score(Y, y_pred).round(2)*100
        coeficient = (linear_regressor.coef_)

        if coeficient > 0:
            Predicted_Trade= 'BUY'
        else:
            Predicted_Trade = 'SELL'
        
        predicted_output.append([Predicted_Market_Fit,Predicted_Trade])

        
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              'Current_Market_Stoploss':Current_Market_Stoploss,
              'Rec1': predicted_output[0][0],
              'Rec1_P': predicted_output[0][1],
              'Rec2': predicted_output[1][0],
              'Rec2_P': predicted_output[1][1],
              'Rec3': predicted_output[2][0],
              'Rec3_P': predicted_output[2][1],
             }
    
    result_output = result_output.append(result, ignore_index = True)

CPU times: user 2min 47s, sys: 30.2 s, total: 3min 17s
Wall time: 3min 17s


In [13]:
result_output.to_csv('Back_Test_Result.csv', header = True, index = False)
result_output = pd.read_csv('Back_Test_Result.csv')

In [16]:
result_output

,Candle_No,Current_Market_Fit,Current_Market,Current_Market_Stoploss,Rec1,Rec1_P,Rec2,Rec2_P,Rec3,Rec3_P
0,6588.0,68.0,Bearish,0.0,97.0,BUY,69.0,SELL,84.0,BUY
1,4779.0,100.0,Bearish,0.0,46.0,BUY,1.0,BUY,20.0,BUY
2,6286.0,97.0,Bearish,0.0,24.0,SELL,35.0,SELL,95.0,BUY
3,1205.0,60.0,Bullish,0.0,83.0,BUY,94.0,SELL,31.0,SELL
4,2823.0,100.0,Bullish,0.0,82.0,BUY,84.0,BUY,2.0,SELL
...,...,...,...,...,...,...,...,...,...,...
995,8119.0,38.0,Bearish,0.0,16.0,BUY,88.0,BUY,99.0,SELL
996,9441.0,98.0,Bullish,0.0,99.0,SELL,3.0,BUY,70.0,SELL
997,1832.0,100.0,Bullish,0.0,83.0,SELL,81.0,SELL,69.0,BUY
998,8420.0,11.0,Bullish,0.0,68.0,BUY,1.0,SELL,74.0,SELL
